<center>
<img src="https://habrastorage.org/web/677/8e1/337/6778e1337c3d4b159d7e99df94227cb2.jpg"/>
## Специализация "Машинное обучение и анализ данных"
<center>Автор материала: программист-исследователь Mail.Ru Group, старший преподаватель Факультета Компьютерных Наук ВШЭ [Юрий Кашницкий](https://yorko.github.io/)

# <center> Capstone проект №1 <br> Идентификация пользователей по посещенным веб-страницам
<img src='http://i.istockimg.com/file_thumbview_approve/21546327/5/stock-illustration-21546327-identification-de-l-utilisateur.jpg'>

# <center>Неделя 5.  Соревнование Kaggle "Catch Me If You Can"

На этой неделе мы вспомним про концепцию стохастического градиентного спуска и опробуем классификатор Scikit-learn SGDClassifier, который работает намного быстрее на больших выборках, чем алгоритмы, которые мы тестировали на 4 неделе. Также мы познакомимся с данными [соревнования](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2) Kaggle по идентификации пользователей и сделаем в нем первые посылки. По итогам этой недели дополнительные баллы получат те, кто попадет в топ-30 публичного лидерборда соревнования.

**В этой части проекта Вам могут быть полезны видеозаписи следующих лекций курса "Обучение на размеченных данных":**
   - [Стохатический градиентный спуск](https://www.coursera.org/learn/supervised-learning/lecture/xRY50/stokhastichieskii-ghradiientnyi-spusk)
   - [Линейные модели. Sklearn.linear_model. Классификация](https://www.coursera.org/learn/supervised-learning/lecture/EBg9t/linieinyie-modieli-sklearn-linear-model-klassifikatsiia)
   
**Также рекомендуется вернуться и просмотреть [задание](https://www.coursera.org/learn/supervised-learning/programming/t2Idc/linieinaia-rieghriessiia-i-stokhastichieskii-ghradiientnyi-spusk) "Линейная регрессия и стохастический градиентный спуск" 1 недели 2 курса специализации.**

### Задание
1. Заполните код в этой тетрадке 
2. Если вы проходите специализацию Яндеса и МФТИ, пошлите тетрадку в соответствующем Peer Review. <br> Если вы проходите курс ODS, выберите ответы в [веб-форме](https://docs.google.com/forms/d/1pLsegkAICL9PzOLyAeH9DmDOBfktte0l8JW75uWcTng). 

In [1]:
from __future__ import division, print_function
# отключим всякие предупреждения Anaconda
import warnings
warnings.filterwarnings('ignore')
from glob import glob
import os
import pickle
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import roc_auc_score
from tqdm import tqdm_notebook
import re
from scipy.sparse import hstack

**Считаем данные [соревнования](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2) в DataFrame train_df и test_df (обучающая и тестовая выборки).**

In [2]:
# Поменяйте на свой путь к данным
# PATH_TO_DATA = r'C:\Users\Дмитрий Мошкин\Documents\GitHub\moshkin\MFTI_Yandex_DA_specialisation\6 course\catch-me-if-you-can'
PATH_TO_DATA = r'C:\Users\MI\Documents\GitHub\moshkin\MFTI_Yandex_DA_specialisation\6 course\catch-me-if-you-can'

In [3]:
train_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'train_sessions.csv'),
                       index_col='session_id')
test_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'test_sessions.csv'),
                      index_col='session_id')

In [4]:
train_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
1,718,2014-02-20 10:02:45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,890,2014-02-22 11:19:50,941.0,2014-02-22 11:19:50,3847.0,2014-02-22 11:19:51,941.0,2014-02-22 11:19:51,942.0,2014-02-22 11:19:51,...,2014-02-22 11:19:51,3847.0,2014-02-22 11:19:52,3846.0,2014-02-22 11:19:52,1516.0,2014-02-22 11:20:15,1518.0,2014-02-22 11:20:16,0
3,14769,2013-12-16 16:40:17,39.0,2013-12-16 16:40:18,14768.0,2013-12-16 16:40:19,14769.0,2013-12-16 16:40:19,37.0,2013-12-16 16:40:19,...,2013-12-16 16:40:19,14768.0,2013-12-16 16:40:20,14768.0,2013-12-16 16:40:21,14768.0,2013-12-16 16:40:22,14768.0,2013-12-16 16:40:24,0
4,782,2014-03-28 10:52:12,782.0,2014-03-28 10:52:42,782.0,2014-03-28 10:53:12,782.0,2014-03-28 10:53:42,782.0,2014-03-28 10:54:12,...,2014-03-28 10:54:42,782.0,2014-03-28 10:55:12,782.0,2014-03-28 10:55:42,782.0,2014-03-28 10:56:12,782.0,2014-03-28 10:56:42,0
5,22,2014-02-28 10:53:05,177.0,2014-02-28 10:55:22,175.0,2014-02-28 10:55:22,178.0,2014-02-28 10:55:23,177.0,2014-02-28 10:55:23,...,2014-02-28 10:55:59,175.0,2014-02-28 10:55:59,177.0,2014-02-28 10:55:59,177.0,2014-02-28 10:57:06,178.0,2014-02-28 10:57:11,0


**Объединим обучающую и тестовую выборки – это понадобится, чтоб вместе потом привести их к разреженному формату.**

In [5]:
train_test_df = pd.concat([train_df, test_df])

In [274]:
train_test_df.columns

Index(['site1', 'site10', 'site2', 'site3', 'site4', 'site5', 'site6', 'site7',
       'site8', 'site9', 'target', 'time1', 'time10', 'time2', 'time3',
       'time4', 'time5', 'time6', 'time7', 'time8', 'time9'],
      dtype='object')

В обучающей выборке видим следующие признаки:
    - site1 – индекс первого посещенного сайта в сессии
    - time1 – время посещения первого сайта в сессии
    - ...
    - site10 – индекс 10-го посещенного сайта в сессии
    - time10 – время посещения 10-го сайта в сессии
    - user_id – ID пользователя
    
Сессии пользователей выделены таким образом, что они не могут быть длинее получаса или 10 сайтов. То есть сессия считается оконченной либо когда пользователь посетил 10 сайтов подряд, либо когда сессия заняла по времени более 30 минут. 

**Посмотрим на статистику признаков.**

Пропуски возникают там, где сессии короткие (менее 10 сайтов). Скажем, если человек 1 января 2015 года посетил *vk.com* в 20:01, потом *yandex.ru* в 20:29, затем *google.com* в 20:33, то первая его сессия будет состоять только из двух сайтов (site1 – ID сайта *vk.com*, time1 – 2015-01-01 20:01:00, site2 – ID сайта  *yandex.ru*, time2 – 2015-01-01 20:29:00, остальные признаки – NaN), а начиная с *google.com* пойдет новая сессия, потому что уже прошло более 30 минут с момента посещения *vk.com*.

In [275]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 253561 entries, 1 to 253561
Data columns (total 21 columns):
site1     253561 non-null int64
time1     253561 non-null object
site2     250098 non-null float64
time2     250098 non-null object
site3     246919 non-null float64
time3     246919 non-null object
site4     244321 non-null float64
time4     244321 non-null object
site5     241829 non-null float64
time5     241829 non-null object
site6     239495 non-null float64
time6     239495 non-null object
site7     237297 non-null float64
time7     237297 non-null object
site8     235224 non-null float64
time8     235224 non-null object
site9     233084 non-null float64
time9     233084 non-null object
site10    231052 non-null float64
time10    231052 non-null object
target    253561 non-null int64
dtypes: float64(9), int64(2), object(10)
memory usage: 42.6+ MB


In [276]:
test_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,site6,time6,site7,time7,site8,time8,site9,time9,site10,time10
session_id,,,,,,,,,,,,,,,,,,,,
1,29,2014-10-04 11:19:53,35.0,2014-10-04 11:19:53,22.0,2014-10-04 11:19:54,321.0,2014-10-04 11:19:54,23.0,2014-10-04 11:19:54,2211.0,2014-10-04 11:19:54,6730.0,2014-10-04 11:19:54,21.0,2014-10-04 11:19:54,44582.0,2014-10-04 11:20:00,15336.0,2014-10-04 11:20:00
2,782,2014-07-03 11:00:28,782.0,2014-07-03 11:00:53,782.0,2014-07-03 11:00:58,782.0,2014-07-03 11:01:06,782.0,2014-07-03 11:01:09,782.0,2014-07-03 11:01:10,782.0,2014-07-03 11:01:23,782.0,2014-07-03 11:01:29,782.0,2014-07-03 11:01:30,782.0,2014-07-03 11:01:53
3,55,2014-12-05 15:55:12,55.0,2014-12-05 15:55:13,55.0,2014-12-05 15:55:14,55.0,2014-12-05 15:56:15,55.0,2014-12-05 15:56:16,55.0,2014-12-05 15:56:17,55.0,2014-12-05 15:56:18,55.0,2014-12-05 15:56:19,1445.0,2014-12-05 15:56:33,1445.0,2014-12-05 15:56:36
4,1023,2014-11-04 10:03:19,1022.0,2014-11-04 10:03:19,50.0,2014-11-04 10:03:20,222.0,2014-11-04 10:03:21,202.0,2014-11-04 10:03:21,3374.0,2014-11-04 10:03:22,50.0,2014-11-04 10:03:22,48.0,2014-11-04 10:03:22,48.0,2014-11-04 10:03:23,3374.0,2014-11-04 10:03:23
5,301,2014-05-16 15:05:31,301.0,2014-05-16 15:05:32,301.0,2014-05-16 15:05:33,66.0,2014-05-16 15:05:39,67.0,2014-05-16 15:05:40,69.0,2014-05-16 15:05:40,70.0,2014-05-16 15:05:40,68.0,2014-05-16 15:05:40,71.0,2014-05-16 15:05:40,167.0,2014-05-16 15:05:44


In [277]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 82797 entries, 1 to 82797
Data columns (total 20 columns):
site1     82797 non-null int64
time1     82797 non-null object
site2     81308 non-null float64
time2     81308 non-null object
site3     80075 non-null float64
time3     80075 non-null object
site4     79182 non-null float64
time4     79182 non-null object
site5     78341 non-null float64
time5     78341 non-null object
site6     77566 non-null float64
time6     77566 non-null object
site7     76840 non-null float64
time7     76840 non-null object
site8     76151 non-null float64
time8     76151 non-null object
site9     75484 non-null float64
time9     75484 non-null object
site10    74806 non-null float64
time10    74806 non-null object
dtypes: float64(9), int64(1), object(10)
memory usage: 13.3+ MB


**В обучающей выборке – 2297 сессий одного пользователя (Alice) и 251264 сессий – других пользователей, не Элис. Дисбаланс классов очень сильный, и смотреть на долю верных ответов (accuracy) непоказательно.**

**Пока для прогноза будем использовать только индексы посещенных сайтов. Индексы нумеровались с 1, так что заменим пропуски на нули.**

In [21]:
train_test_df_sites = train_test_df[['site%d' % i for i in range(1, 11)]].fillna(0).astype('int')

In [22]:
train_test_df_sites.head(10)

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
session_id,,,,,,,,,,
1,718,0,0,0,0,0,0,0,0,0
2,890,941,3847,941,942,3846,3847,3846,1516,1518
3,14769,39,14768,14769,37,39,14768,14768,14768,14768
4,782,782,782,782,782,782,782,782,782,782
5,22,177,175,178,177,178,175,177,177,178
6,570,21,570,21,21,0,0,0,0,0
7,803,23,5956,17513,37,21,803,17514,17514,17514
8,22,21,29,5041,14422,23,21,5041,14421,14421
9,668,940,942,941,941,942,940,23,21,22


**Создайте разреженные матрицы *X_train_sparse* и *X_test_sparse* аналогично тому, как мы это делали ранее. Используйте объединенную матрицу *train_test_df_sites*, потом разделите обратно на обучающую и тестовую части.**

Обратите внимание на то, что в  сессиях меньше 10 сайтов  у нас остались нули, так что первый признак (сколько раз попался 0) по смыслу отличен от остальных (сколько раз попался сайт с индексом $i$). Поэтому первый столбец разреженной матрицы надо будет удалить.

**Выделите в отдельный вектор *y* ответы на обучающей выборке.**

In [18]:
def sparsecreator(X):
    '''takes matrix and returns sparse matrix alternative'''
    
    indptr = [0]
    indices = []
    vocabulary = {}
    data = []
    for row in X:
        for column in row:
            index = vocabulary.setdefault(column, column)
            indices.append(index)
            data.append(1)
        indptr.append(len(indices))
    return csr_matrix((data, indices, indptr), dtype=int)[:, 1:] # deleting 0 index

In [24]:
train_test_sparse = sparsecreator(train_test_df_sites.values)
train_test_sparse = train_test_sparse[:, 1:]
X_train_sparse = train_test_sparse[:253561]
X_test_sparse = train_test_sparse[253561:]
y = train_df.target.values

**<font color='red'>Вопрос 1. </font> Выведите размерности матриц *X_train_sparse* и *X_test_sparse* – 4 числа на одной строке через пробел: число строк и столбцов матрицы *X_train_sparse*, затем число строк и столбцов матрицы *X_test_sparse*.**

In [25]:
for i in X_train_sparse.shape + X_test_sparse.shape:
    print(str(i), end=' ')

253561 48370 82797 48370 

**Сохраним в pickle-файлы объекты *X_train_sparse*, *X_test_sparse* и *y* (последний – в файл *kaggle_data/train_target.pkl*).**

In [26]:
with open(os.path.join(PATH_TO_DATA, 'X_train_sparse.pkl'), 'wb') as X_train_sparse_pkl:
    pickle.dump(X_train_sparse, X_train_sparse_pkl, protocol=2)
with open(os.path.join(PATH_TO_DATA, 'X_test_sparse.pkl'), 'wb') as X_test_sparse_pkl:
    pickle.dump(X_test_sparse, X_test_sparse_pkl, protocol=2)
with open(os.path.join(PATH_TO_DATA, 'train_target.pkl'), 'wb') as train_target_pkl:
    pickle.dump(y, train_target_pkl, protocol=2)

**Разобьем обучающую выборку на 2 части в пропорции 7/3, причем не перемешивая. Исходные данные упорядочены по времени, тестовая выборка по времени четко отделена от обучающей, это же соблюдем и здесь.**

In [27]:
train_share = int(.7 * X_train_sparse.shape[0])
X_train, y_train = X_train_sparse[:train_share, :], y[:train_share]
X_valid, y_valid  = X_train_sparse[train_share:, :], y[train_share:]

**Создайте объект `sklearn.linear_model.SGDClassifier` с логистической функцией потерь и параметром *random_state*=17. Остальные параметры оставьте по умолчанию, разве что *n_jobs*=-1 никогда не помешает. Обучите  модель на выборке `(X_train, y_train)`.**

In [28]:
sgd_logit = SGDClassifier(loss='log', random_state=17, n_jobs=-1)
sgd_logit.fit(X_train, y_train)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=-1, penalty='l2',
       power_t=0.5, random_state=17, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False)

**Сделайте прогноз в виде предсказанных вероятностей того, что это сессия Элис, на отложенной выборке *(X_valid, y_valid)*.**

In [29]:
logit_valid_pred_proba = sgd_logit.predict_proba(X_valid)

**<font color='red'>Вопрос 2. </font> Посчитайте ROC AUC логистической регрессии, обученной с помощью стохастического градиентного спуска, на отложенной выборке. Округлите до 3 знаков после разделителя.**

In [30]:
round(roc_auc_score(y_valid, logit_valid_pred_proba[:, 1]), 3)

0.934

In [31]:
logit_valid_pred_proba

array([[9.22532691e-01, 7.74673091e-02],
       [9.99659621e-01, 3.40379118e-04],
       [9.99868953e-01, 1.31046863e-04],
       ...,
       [9.86717995e-01, 1.32820047e-02],
       [9.99944253e-01, 5.57466393e-05],
       [9.88745587e-01, 1.12544131e-02]])

**Сделайте прогноз в виде предсказанных вероятностей отнесения к классу 1 для тестовой выборки с помощью той же *sgd_logit*, обученной уже на всей обучающей выборке (а не на 70%).**

In [45]:
%%time
sgd_logit = SGDClassifier(loss='log', random_state=17, n_jobs=-1)
sgd_logit.fit(X_train_sparse, y)
logit_test_pred_proba = sgd_logit.predict_proba(X_test_sparse)

Wall time: 285 ms


**Запишите ответы в файл и сделайте посылку на Kaggle. Дайте своей команде (из одного человека) на Kaggle говорящее название – по шаблону "[YDF & MIPT] Coursera_Username", чтоб можно было легко идентифицировать Вашу посылку на [лидерборде](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2/leaderboard/public).**

**Результат, который мы только что получили, соответствует бейзлайну "SGDCLassifer" на лидерборде, задача на эту неделю – как минимум его побить.**

In [26]:
def write_to_submission_file(predicted_labels, out_file,
                             target='target', index_label="session_id"):
    # turn predictions into data frame and save as csv file
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [47]:
write_to_submission_file(logit_test_pred_proba[:, 1], 'w5_a1.csv')

## Обучение логрегрессии на выборке с дополнительными признаками

Для начала создадим функцию, которая посчитает нам четыре новых признака.

На вход подадим train_test_df из конкатенированных ранее матриц train_sessions.csv и test_sessions.csv

In [9]:
def feature_adder(frame):
    
    frame.drop('target', axis=1, inplace=True)
    session_timespans = []
    days_of_week = []
    start_hours = []
    unique_sites_list = []
    for idx in tqdm_notebook(range(frame.shape[0])):
        timestamps = frame.iloc[idx, 10:20].apply(pd.to_datetime)
        timemax = timestamps.max()
        timemin = timestamps.min()
        unique_sites = len(np.unique(train_test_df.iloc[idx, 0:10].dropna()))

        session_timespans.append((timemax - timemin).seconds)
        days_of_week.append(timemin.dayofweek)
        start_hours.append(timemin.hour)
        unique_sites_list.append(unique_sites)
        
    frame['session_timespan'] = session_timespans
    frame['day_of_week'] = days_of_week
    frame['start_hour'] = start_hours
    frame['unique_sites'] = unique_sites_list
    frame.fillna(0, inplace=True)  
    frame.iloc[:, 0:10] = frame.iloc[:, 0:10].astype('int') 
    
    return frame

In [10]:
train_test_df_0 = train_test_df[:]
X = feature_adder(train_test_df_0)

KeyboardInterrupt: 

In [11]:
# Серриализуем получившися массив

with open(os.path.join(PATH_TO_DATA, 
                       'kaggle_dataset_new_old_features.pkl'), 'wb') as f:
    pickle.dump(X, f)

Теперь преобразуем наши признаки с именами сайтов в вещественные (как мы это и делали с помощью нашей функции для создания 
sparse matrix)

In [49]:
X_sparse = sparsecreator(X.iloc[:, :10].values)
X_sparse = X_sparse[:, 1:]

Далее преобразуем в том же ключе новые признаки, но уже с использованием pd.get_dummies

In [50]:
dummies = pd.get_dummies(X.iloc[:, -4:], sparse=True, columns=['start_hour', 'unique_sites', 'day_of_week','session_timespan'])

Соединим обе полученые матрицы по оси столбцов

In [51]:
X_sparse_dum = hstack([X_sparse, csr_matrix(dummies)], format='csr')

Далее привычным образом разобьем данные на обучающую и тестовую подвыборки, ничего при этом не перемешивая.

In [52]:
X_train_sparse = X_sparse_dum[:253561]
X_test_sparse = X_sparse_dum[253561:]
y = train_df.target.values

Разобьем X_test_sparse на обучение и тест, обучим классификатор и посчитаем площадь под ROC-кривой

In [53]:
train_share = int(.7 * X_train_sparse.shape[0])
X_train, y_train = X_train_sparse[:train_share, :], y[:train_share]
X_valid, y_valid  = X_train_sparse[train_share:, :], y[train_share:]

In [54]:
sgd_logit_mini = SGDClassifier(loss='log', random_state=17, n_jobs=-1)
sgd_logit_mini.fit(X_train, y_train)
logit_valid_pred_proba = sgd_logit_mini.predict_proba(X_valid)
round(roc_auc_score(y_valid, logit_valid_pred_proba[:, 1]), 3)

0.971

Результат выглядит неплохо. Попробуем обучить классификатор на полной выборке X_train_sparse и получить вектор предсказаний меток для X_test_sparse.

Отправим данные на kaggle и посмотрим на результат.

In [48]:
sgd_logit = SGDClassifier(loss='log', random_state=17, n_jobs=-1)
sgd_logit.fit(X_train_sparse, y)
logit_test_pred_proba = sgd_logit.predict_proba(X_test_sparse)

In [345]:
write_to_submission_file(logit_test_pred_proba[:, 1], r'C:\Users\Дмитрий Мошкин\Documents\GitHub\moshkin\MFTI_Yandex_DA_specialisation\6 course\w5_a3.csv')

Результат на kaggle 0.93562, что превосходит бейзлайн 'Logit +3 features' с его результатом 0.92784

Результат можно найти в лидербоард как [YDF & MIPT] Coursera_Dmi2ry

## Критерии оценки работы (только для Peer Review в специализации):
- Правильные ли получились размерности матриц в п. 1? (max. 2 балла)
- Правильным ли получилось значения ROC AUC в п. 2? (max. 4 балла)
- Побит ли бенчмарк "sgd_logit_benchmark.csv" на публичной части рейтинга в соревновании Kaggle? (max. 2 балла)
- Побит ли бенчмарк "Logit +3 features" на публичной части рейтинга в соревновании Kaggle? (max. 2 балла)

## Пути улучшения
На этой неделе дается много времени на соревнование. Не забывайте вносить хорошие идеи, к которым Вы пришли по ходу соревнования, в описание финального проекта (`html`, `pdf` или `ipynb`). Это только в случае, если вы проходите специализацию.
Что можно попробовать:
 - Использовать ранее построенные признаки для улучшения модели (проверить их можно на меньшей выборке по 150 пользователям, отделив одного из пользователей от остальных – это быстрее)
 - Настроить параметры моделей (например, коэффициенты регуляризации)
 - Если позволяют мощности (или хватает терпения), можно попробовать смешивание (блендинг) ответов бустинга и линейной модели. [Вот](http://mlwave.com/kaggle-ensembling-guide/) один из самых известных тьюториалов по смешиванию ответов алгоритмов, также хороша [статья](https://alexanderdyakonov.wordpress.com/2017/03/10/cтекинг-stacking-и-блендинг-blending) Александра Дьяконова
 - Обратите внимание, что в соревновании также даны исходные данные о посещенных веб-страницах Элис и остальными 1557 пользователями (*train.zip*). По этим данным можно сформировать свою обучающую выборку. 

На 6 неделе мы пройдем большой тьюториал по Vowpal Wabbit и попробуем его в деле, на данных соревнования.

## Дополнительные фичи

В рамках идентифицирования типа деятельности пользователя (отдых\работа) добавим такие фичи, как флажки посещения фейсбука, почты и ютьюб. 

In [6]:
def feature_adder(frame):
    
    frame.drop('target', axis=1, inplace=True)
    session_timespans = []
    days_of_week = []
    start_hours = []
    unique_sites_list = []
    
    avgtime = []
    avgtimefeature = []
    facebookflag = 0
    facebookfeature = []
    maillist = []
    mailflag = 0
    youtubelist = []
    youtubeflag = 0
    
    for idx in tqdm_notebook(range(frame.shape[0])):
        timestamps = frame.iloc[idx, 10:20].apply(pd.to_datetime)
        timemax = timestamps.max()
        timemin = timestamps.min()
        session = train_test_df.iloc[idx, 0:10].dropna().astype('int')
        unique_sites = len(np.unique(session))
        
        if 4 in session:
            mailflag = 1
        if 5 in session:
            facebookflag = 1
        if 1 in session:
            youtubeflag = 1
            

        facebookfeature.append(facebookflag)
        facebookflag = 0
        maillist.append(mailflag)
        mailflag = 0
        youtubelist.append(youtubeflag)
        youtubeflag = 0
        
        session_timespans.append((timemax - timemin).seconds)
        days_of_week.append(timemin.dayofweek)
        start_hours.append(timemin.hour)
        unique_sites_list.append(unique_sites)
        
    frame['session_timespan'] = session_timespans
    frame['day_of_week'] = days_of_week
    frame['start_hour'] = start_hours
    frame['unique_sites'] = unique_sites_list
    
    frame['mail'] = maillist
    frame['facebook'] = facebookfeature
#     frame['top30_av_time'] = avgtimefeature
    frame['youtube'] = youtubelist
    
    frame.fillna(0, inplace=True)  
    frame.iloc[:, 0:10] = frame.iloc[:, 0:10].astype('int') 
    
    return frame

In [7]:
train_test_df_0 = train_test_df[:]
X = feature_adder(train_test_df_0)

In [8]:
with open(os.path.join(PATH_TO_DATA, 
                       'kaggle_dataset_additional_features_for_report.pkl'), 'wb') as f:
    pickle.dump(X, f)

In [15]:
X.head()

,site1,site10,site2,site3,site4,site5,site6,site7,site8,site9,...,time7,time8,time9,session_timespan,day_of_week,start_hour,unique_sites,mail,facebook,youtube
session_id,,,,,,,,,,,,,,,,,,,,,
1,718,0,0,0,0,0,0,0,0,0,...,0,0,0,0,3,10,1,0,0,0
2,890,1518,941,3847,941,942,3846,3847,3846,1516,...,2014-02-22 11:19:52,2014-02-22 11:19:52,2014-02-22 11:20:15,26,5,11,7,0,0,0
3,14769,14768,39,14768,14769,37,39,14768,14768,14768,...,2013-12-16 16:40:20,2013-12-16 16:40:21,2013-12-16 16:40:22,7,0,16,4,0,0,0
4,782,782,782,782,782,782,782,782,782,782,...,2014-03-28 10:55:12,2014-03-28 10:55:42,2014-03-28 10:56:12,270,4,10,1,0,0,0
5,22,178,177,175,178,177,178,175,177,177,...,2014-02-28 10:55:59,2014-02-28 10:55:59,2014-02-28 10:57:06,246,4,10,4,0,0,0


In [19]:
X_sparse = sparsecreator(X.iloc[:, :10].values)
X_sparse = X_sparse[:, 1:]

In [16]:
dummies = pd.get_dummies(X.iloc[:, -7:], sparse=True, 
                         columns=['start_hour', 'unique_sites', 'day_of_week',
                                  'session_timespan', 'mail', 'facebook', 'youtube'])

In [20]:
X_sparse_dum = hstack([X_sparse, csr_matrix(dummies)], format='csr')

In [21]:
X_train_sparse = X_sparse_dum[:253561]
X_test_sparse = X_sparse_dum[253561:]
y = train_df.target.values

In [22]:
train_share = int(.7 * X_train_sparse.shape[0])
X_train, y_train = X_train_sparse[:train_share, :], y[:train_share]
X_valid, y_valid  = X_train_sparse[train_share:, :], y[train_share:]

In [23]:
sgd_logit_mini = SGDClassifier(loss='log', random_state=17, n_jobs=-1)
sgd_logit_mini.fit(X_train, y_train)
logit_valid_pred_proba = sgd_logit_mini.predict_proba(X_valid)
round(roc_auc_score(y_valid, logit_valid_pred_proba[:, 1]), 3)

0.973

Результат получился немного выше.

In [24]:
sgd_logit = SGDClassifier(loss='log', random_state=17, n_jobs=-1)
sgd_logit.fit(X_train_sparse, y)
logit_test_pred_proba = sgd_logit.predict_proba(X_test_sparse)

In [27]:
write_to_submission_file(logit_test_pred_proba[:, 1], os.path.join(PATH_TO_DATA, 'w5_a4.csv'))

Результат на kaggle 0.93734, что превосходит предыдущий результат 0.93562.